# Análisis de fragmentación de tablas/índices

<span style="font-size: 14px;">Para ver si una BBDD necesita que se apliquen técnicas de defragmentación, esto te ayudaría:</span>

- **Fragmentación de índices**: La fragmentación es un indicador de que los índices pueden necesitar mantenimiento. Puedes usar la función sys.dm\_db\_index\_physical\_stats para obtener información sobre la fragmentación de índices en tu base de datos.
    
- <span style="font-size: 14px;"><b>Estadísticas desactualizadas</b>: Las estadísticas desactualizadas pueden llevar a SQL Server a tomar decisiones de optimización subóptimas. Puedes usar la función sys.dm_db_stats_properties para ver cuándo se actualizaron por última vez las estadísticas.</span>
    
- <span style="font-size: 14px;"><b>Rendimiento de la consulta</b>: Si las consultas se están volviendo más lentas con el tiempo, puede ser un indicador de que los índices necesitan mantenimiento.</span>
    

  

Normalmente las operaciones de defragmentación se agendan con SQL Server Agent. Busca si hay algún job que nombre mantenimiento, maintenance, index\_rebuild, etc. que muestre indicios de que hay algo implementado.

  

Si no localizas nada, probablemente no haya estrategia de mantenimiento implementada.

# Fragmentación de índices

In [ ]:
SELECT 
    dbschemas.[name] as 'Schema',
    dbtables.[name] as 'Table',
    dbindexes.[name] as 'Index',
    indexstats.avg_fragmentation_in_percent,
    indexstats.page_count
FROM 
    sys.dm_db_index_physical_stats (DB_ID(), NULL, NULL, NULL, NULL) AS indexstats
INNER JOIN 
    sys.tables dbtables on dbtables.[object_id] = indexstats.[object_id]
INNER JOIN 
    sys.schemas dbschemas on dbtables.[schema_id] = dbschemas.[schema_id]
INNER JOIN 
    sys.indexes AS dbindexes ON dbindexes.[object_id] = indexstats.[object_id]
    AND indexstats.index_id = dbindexes.index_id
WHERE   
    indexstats.database_id = DB_ID()
ORDER BY 
    indexstats.avg_fragmentation_in_percent desc


## Documentación dm\_db\_index\_physical\_stats

Enlace --\> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">(</span>[sys.dm\_db\_index\_physical\_stats (Transact-SQL) - SQL Server | Microsoft Learn](https:\learn.microsoft.com\es-es\sql\relational-databases\system-dynamic-management-views\sys-dm-db-index-physical-stats-transact-sql?view=sql-server-ver16)<span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">)</span>

Revisar con detenimiento los argumentos y opciones. Permiten elegir qué objetos analizar y en modo de análisis (argumento mode). Es bueno probar las opciones en un entorno seguro y con volumen suficiente de datos, para tomar conciencia real de las posibilidades.

Un fallo habitual de falta de comprensión es hacer CROSS JOIN con la UDF pasando un nombre de tabla, pensando que sólo va a leer ese objeto. Pruebalo y saca conclusiones.

Revisar con interés los ejemplos de la documentación ([sys.dm\_db\_index\_physical\_stats (Transact-SQL) - SQL Server | Microsoft Learn](https://learn.microsoft.com/es-es/sql/relational-databases/system-dynamic-management-views/sys-dm-db-index-physical-stats-transact-sql?view=sql-server-ver16#examples)).

Conceptos a revisar: fragmentación lógica, fragmentación de extensiones,

# Estadisticas desactualizadas

In [ ]:
SELECT 
    OBJECT_NAME(s.object_id) AS ObjectName,
    i.name AS IndexName,
    s.name AS StatName,
    STATS_DATE(s.object_id, s.stats_id) AS LastUpdated,
    sp.modification_counter AS ModificationCounter
FROM 
    sys.stats AS s 
JOIN 
    sys.indexes AS i ON s.object_id = i.object_id AND s.stats_id = i.index_id
CROSS APPLY 
    sys.dm_db_stats_properties(s.object_id, s.stats_id) AS sp
WHERE 
    sp.modification_counter > 0
ORDER BY 
    sp.modification_counter DESC;


## Documentación dm\_db\_stats\_properties
Enlace --\> ([sys.dm\_db\_stats\_properties (Transact-SQL) - SQL Server | Microsoft Learn](https:\learn.microsoft.com\en-us\sql\relational-databases\system-dynamic-management-views\sys-dm-db-stats-properties-transact-sql?view=sql-server-ver16)).
Revisar ejemplos para profundizar: [sys.dm\_db\_stats\_properties (Transact-SQL) - SQL Server | Microsoft Learn](https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-db-stats-properties-transact-sql?view=sql-server-ver16)  
Si necesitas estadisticas por cada columna de la base de datos, revisa este enlace de sys.stats: [sys.stats (Transact-SQL) - SQL Server | Microsoft Learn](https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-stats-transact-sql?view=sql-server-ver16)